<a href="https://colab.research.google.com/github/Radhikaagal/projectfacerecognition/blob/master/test1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
 

In [0]:
import patoolib
patoolib.extract_archive("facedataset.tar.gz", outdir=".")


patool: Extracting facedataset.tar.gz ...
patool: running /bin/tar --extract -z --file facedataset.tar.gz --directory .
patool: ... facedataset.tar.gz extracted to `.'.


'.'

In [0]:
import os
import pickle
import cv2
import matplotlib.pyplot as plt
import numpy as np

def create_data():
  x=[]
  y=[]
  label=0
  
  Base_dir = os.path.dirname(os.path.abspath('__file__'))
  data_dir = os.path.join(Base_dir,"facedataset")
  
  for img_folder in os.listdir(data_dir):
      image_file=os.path.join(data_dir,img_folder)
      for file in os.listdir(image_file):
          img_path=os.path.join(image_file,file)
          img=cv2.imread(img_path,cv2.IMREAD_GRAYSCALE)/256
          img=cv2.resize(img,(32,32))
          img=img.reshape(32,32,1)
          x.append(img)
          y.append(label)
      label=label+1
  return x,y
          
x,y=create_data()
np.save("xtrain.npy",x)
np.save("ytrain.npy",y)

In [0]:
import numpy as np
x=np.load("xtrain.npy")
y=np.load("ytrain.npy")
t_g = [x[np.where(y==i)[0]] for i in np.unique(y)] # split the images of diffrent person
train = []
# Function to check similarity
def check_similarity(v):
    el=[]
    el.append(x[v[0]])
    el.append(x[v[1]])
    if y[v[0]]!=y[v[1]]:
      el.append(0)
    else:
      el.append(1)
    train.append(el)
    
# Function to generate only similar pair
def generate_similar(t_g):
   for i in range(0,50):
      for j in range(800):
        v=np.random.choice(len(t_g[i]),2)
        if v[0]!=v[1]:
          el2=[]
          el2.append(t_g[i][v[0]])
          el2.append(t_g[i][v[1]])
          el2.append(1)
          train.append(el2)

# Function to generate pair of images with labels
def generate_pair():
  for i in range(1,50):
    for j in range(1200): 
      v=np.random.choice(15*i,2)
      if v[0]!=v[1]:
        check_similarity(v)   
  generate_similar(t_g)
generate_pair()
t=np.array(train)
t.shape
np.save("pair_data.npy",t)
m=np.where(t[:,-1]==1)
print(len(t),len(m[0]))

    


95867 42474


In [0]:
from random import shuffle
Data=np.load("pair_data.npy")
shuffle(Data)
Y_data=Data[:,-1]
X_data=Data[:,:-1]
X_train_data=X_data[:-38280]
Y_train_data=Y_data[:-38280]

X_test_data=X_data[-38280:-19140]
X_test_data=Y_data[-38280:-19140]

X_val_data=X_data[-19140:]
Y_val_data=Y_data[-19140:]

# print(len(train_data),len(test_data) , len(val_data))

[array([[[0.13334064],
        [0.14453125],
        [0.1444316 ],
        [0.12967355],
        [0.33953478],
        [0.20665257],
        [0.16418208],
        [0.14990233],
        [0.13348015],
        [0.10183155],
        [0.13002232],
        [0.14103355],
        [0.11889149],
        [0.12832827],
        [0.14120298],
        [0.14693276],
        [0.17620975],
        [0.17654855],
        [0.27711457],
        [0.23282043],
        [0.27835028],
        [0.28266506],
        [0.2111966 ],
        [0.39827811],
        [0.25676618],
        [0.26275524],
        [0.30698941],
        [0.28211698]],

       [[0.11698819],
        [0.15764508],
        [0.17245296],
        [0.32463728],
        [0.17332   ],
        [0.17514351],
        [0.16646405],
        [0.12693319],
        [0.14751077],
        [0.12475088],
        [0.12416295],
        [0.15232381],
        [0.16303612],
        [0.19801302],
        [0.15050024],
        [0.13146719],
        [0.17409717],
       

In [0]:
import numpy as np
import tensorflow as tf
import keras
from keras.engine.input_layer import Input
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Lambda
from keras.layers import Conv2D, MaxPooling2D


def create_siamese_model(input_shape):
  first_input = Input(input_shape)
  second_input = Input(input_shape)
  
  model= Sequential()
  model.add(Conv2D(32,(5,5),activation = 'relu', input_shape= input_shape))
  model.add(MaxPooling2D(pool_size=(2,2)))
  
  model.add(Conv2D(64,(3,3),activation = 'relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))
  
  model.add(Conv2D(128,(2,2),activation = 'relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))
  
  model.add(Flatten())
  
  model.add(Dense(128, activation='sigmoid'))
  
  encoded_first = model(first_input)
  encoded_second = model(second_input)
  
  L1_layer= Lambda(lambda f:K.abs(f[0]-f[1]))
  L1_distance = L1_layer([encoded_first,encoded_second])
  
  pred = Dense(1,activation = 'sigmoid')(L1_distance)
  
  siamese_network = Model(inputs=[first_input, second_input], outputs=pred)
  
  return siamese_network

mod= create_siamese_model((32,32,1))
mod.summary() 

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           (None, 32, 32, 1)    0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           (None, 32, 32, 1)    0                                            
__________________________________________________________________________________________________
sequential_6 (Sequential)       (None, 128)          117888      input_11[0][0]                   
                                                                 input_12[0][0]                   
__________________________________________________________________________________________________
lambda_4 (Lambda)               (None, 128)          0           sequential_6[1][0]               
          